In [1]:
import torchvision.models as models
import pydicom
from torch.utils.data import Dataset,DataLoader
import os
import pylab
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import torch
import cv2
from openpyxl import Workbook
from preprocessing import bsb_window

In [2]:
'''
0->硬膜外出血 epidural: 223
1->健康 healthy: 977
2->腦實質性出血 intraparenchymal: 807
3->腦室內出血 intraventricular: 712
4->蛛網膜下腔出血 subarachnoid: 717
5->硬膜下出血 subdural: 794
'''

class_dict={
    0:'epidural',
    1:'healthy',
    2:'intraparenchymal',
    3:'intraventricular',
    4:'subarachnoid',
    5:'subdural'
}

In [3]:
device = torch.cuda.current_device()
print('device:',device)

device: 0


In [4]:
class ICHDataset(Dataset):
    def __init__(self,root,train=True):
        self.train=train
        self.labels=[]
        self.filename_list=[]
        
        if train:
            all_dirs=os.listdir(root)

            label=0
            for dir in all_dirs:
                path=os.path.join(root,dir)
                all_files=os.listdir(path)
                for i,file in enumerate(all_files):
                    self.filename_list.append(os.path.join(path,file))
                    self.labels.append(label)
                label+=1
    
        if not train:
            all_files=os.listdir(root)
            for file in all_files:
                self.filename_list.append(os.path.join(root,file))
            
            
    def __getitem__(self,idx):
        filename=self.filename_list[idx]
        dcm=pydicom.read_file(filename)
        data=bsb_window(dcm)
        data=cv2.resize(data,(512,512))
        
#         pixel_arr=pydicom.read_file(filename).pixel_array
#         pixel_arr=cv2.resize(pixel_arr,(512,512))
#         pixel_arr=np.float32(pixel_arr)
#         pixel_arr=cv2.cvtColor(pixel_arr, cv2.COLOR_GRAY2BGR)
        transform = transforms.Compose([            
            transforms.ToTensor()
        ])
#         data=transform(pixel_arr)
        data=transform(data)
        data=data.type(torch.cuda.FloatTensor)
        if self.train:
            target=self.labels[idx]
            return data,target
        else:
            return data
        
    def __len__(self):
        return len(self.filename_list)
                


In [9]:
epochs=50
lr=0.01
batch_size=16
criterion=nn.CrossEntropyLoss()
num_classes=6

In [6]:
trainset=ICHDataset('TrainingData')
train_dataloader=DataLoader(trainset,batch_size=batch_size,shuffle=True)

In [15]:
print(trainset[1000][1])

1


In [8]:
#Train
def train(model,dataloader,optimizer):
    for epoch in range(epochs):
        correct=0.0
        running_loss=0.0 
        for times,data in enumerate(dataloader):
            optimizer.zero_grad()
            inputs,labels=data
            inputs,labels=inputs.to(device),labels.to(device)
            outputs=model(inputs)
            _, predict = torch.max(outputs.data, 1)
            correct+=(predict==labels).sum().item()

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            print('[%d/%d] [%d/%d] loss: %.3f' % (epoch+1, epochs, times, len(trainset)/batch_size, running_loss))
        
        accuracy=correct/len(trainset)*100
        print('Train accuracy: %.3f%%' % accuracy)


In [11]:
resnet18 = models.resnet18()
resnet18.fc=nn.Linear(512,num_classes)
resnet18=resnet18.to(device)
optimizer=optim.SGD(resnet18.parameters(),lr=lr,momentum=0.9,weight_decay=0.0005)
train(resnet18,train_dataloader,optimizer)

[1/50] [0/375] loss: 1.801
[1/50] [1/375] loss: 3.737
[1/50] [2/375] loss: 5.557
[1/50] [3/375] loss: 7.283
[1/50] [4/375] loss: 9.555
[1/50] [5/375] loss: 11.497
[1/50] [6/375] loss: 13.133
[1/50] [7/375] loss: 15.071
[1/50] [8/375] loss: 17.027
[1/50] [9/375] loss: 19.163
[1/50] [10/375] loss: 20.991
[1/50] [11/375] loss: 23.307
[1/50] [12/375] loss: 25.193
[1/50] [13/375] loss: 27.535
[1/50] [14/375] loss: 29.266
[1/50] [15/375] loss: 31.119
[1/50] [16/375] loss: 33.319
[1/50] [17/375] loss: 35.667
[1/50] [18/375] loss: 37.849
[1/50] [19/375] loss: 39.636
[1/50] [20/375] loss: 41.355
[1/50] [21/375] loss: 44.051
[1/50] [22/375] loss: 46.481
[1/50] [23/375] loss: 48.514
[1/50] [24/375] loss: 50.446
[1/50] [25/375] loss: 52.304
[1/50] [26/375] loss: 54.501
[1/50] [27/375] loss: 57.212
[1/50] [28/375] loss: 59.408
[1/50] [29/375] loss: 61.775
[1/50] [30/375] loss: 64.290
[1/50] [31/375] loss: 66.459
[1/50] [32/375] loss: 68.677
[1/50] [33/375] loss: 71.287
[1/50] [34/375] loss: 73.516


[1/50] [270/375] loss: 497.319
[1/50] [271/375] loss: 499.046
[1/50] [272/375] loss: 500.583
[1/50] [273/375] loss: 502.293
[1/50] [274/375] loss: 503.829
[1/50] [275/375] loss: 505.381
[1/50] [276/375] loss: 507.250
[1/50] [277/375] loss: 509.182
[1/50] [278/375] loss: 510.947
[1/50] [279/375] loss: 512.519
[1/50] [280/375] loss: 514.172
[1/50] [281/375] loss: 515.854
[1/50] [282/375] loss: 517.575
[1/50] [283/375] loss: 519.372
[1/50] [284/375] loss: 520.945
[1/50] [285/375] loss: 522.720
[1/50] [286/375] loss: 524.316
[1/50] [287/375] loss: 526.112
[1/50] [288/375] loss: 527.664
[1/50] [289/375] loss: 529.287
[1/50] [290/375] loss: 530.999
[1/50] [291/375] loss: 532.678
[1/50] [292/375] loss: 534.449
[1/50] [293/375] loss: 536.390
[1/50] [294/375] loss: 538.233
[1/50] [295/375] loss: 540.026
[1/50] [296/375] loss: 541.647
[1/50] [297/375] loss: 543.292
[1/50] [298/375] loss: 544.985
[1/50] [299/375] loss: 546.636
[1/50] [300/375] loss: 548.388
[1/50] [301/375] loss: 550.369
[1/50] [

[2/50] [165/375] loss: 280.211
[2/50] [166/375] loss: 282.010
[2/50] [167/375] loss: 283.669
[2/50] [168/375] loss: 285.365
[2/50] [169/375] loss: 287.055
[2/50] [170/375] loss: 289.038
[2/50] [171/375] loss: 290.427
[2/50] [172/375] loss: 291.981
[2/50] [173/375] loss: 293.885
[2/50] [174/375] loss: 295.714
[2/50] [175/375] loss: 297.355
[2/50] [176/375] loss: 299.228
[2/50] [177/375] loss: 300.884
[2/50] [178/375] loss: 302.918
[2/50] [179/375] loss: 304.611
[2/50] [180/375] loss: 305.992
[2/50] [181/375] loss: 307.671
[2/50] [182/375] loss: 309.307
[2/50] [183/375] loss: 311.222
[2/50] [184/375] loss: 312.881
[2/50] [185/375] loss: 315.131
[2/50] [186/375] loss: 316.950
[2/50] [187/375] loss: 318.443
[2/50] [188/375] loss: 320.227
[2/50] [189/375] loss: 321.815
[2/50] [190/375] loss: 323.421
[2/50] [191/375] loss: 325.073
[2/50] [192/375] loss: 326.832
[2/50] [193/375] loss: 328.669
[2/50] [194/375] loss: 330.316
[2/50] [195/375] loss: 331.748
[2/50] [196/375] loss: 333.375
[2/50] [

[3/50] [58/375] loss: 97.307
[3/50] [59/375] loss: 98.851
[3/50] [60/375] loss: 100.184
[3/50] [61/375] loss: 101.751
[3/50] [62/375] loss: 103.299
[3/50] [63/375] loss: 105.139
[3/50] [64/375] loss: 106.778
[3/50] [65/375] loss: 108.539
[3/50] [66/375] loss: 110.284
[3/50] [67/375] loss: 111.936
[3/50] [68/375] loss: 113.612
[3/50] [69/375] loss: 115.190
[3/50] [70/375] loss: 116.735
[3/50] [71/375] loss: 118.221
[3/50] [72/375] loss: 119.896
[3/50] [73/375] loss: 121.224
[3/50] [74/375] loss: 122.780
[3/50] [75/375] loss: 124.560
[3/50] [76/375] loss: 125.998
[3/50] [77/375] loss: 127.913
[3/50] [78/375] loss: 129.407
[3/50] [79/375] loss: 131.035
[3/50] [80/375] loss: 132.870
[3/50] [81/375] loss: 134.601
[3/50] [82/375] loss: 136.492
[3/50] [83/375] loss: 138.182
[3/50] [84/375] loss: 140.121
[3/50] [85/375] loss: 141.877
[3/50] [86/375] loss: 143.381
[3/50] [87/375] loss: 144.732
[3/50] [88/375] loss: 146.409
[3/50] [89/375] loss: 148.230
[3/50] [90/375] loss: 149.822
[3/50] [91/3

[3/50] [324/375] loss: 532.353
[3/50] [325/375] loss: 534.289
[3/50] [326/375] loss: 535.752
[3/50] [327/375] loss: 537.239
[3/50] [328/375] loss: 538.709
[3/50] [329/375] loss: 540.308
[3/50] [330/375] loss: 542.145
[3/50] [331/375] loss: 543.980
[3/50] [332/375] loss: 545.662
[3/50] [333/375] loss: 547.191
[3/50] [334/375] loss: 548.915
[3/50] [335/375] loss: 550.571
[3/50] [336/375] loss: 552.061
[3/50] [337/375] loss: 553.690
[3/50] [338/375] loss: 555.035
[3/50] [339/375] loss: 556.574
[3/50] [340/375] loss: 558.223
[3/50] [341/375] loss: 559.758
[3/50] [342/375] loss: 561.225
[3/50] [343/375] loss: 562.830
[3/50] [344/375] loss: 564.365
[3/50] [345/375] loss: 565.718
[3/50] [346/375] loss: 567.301
[3/50] [347/375] loss: 568.990
[3/50] [348/375] loss: 570.499
[3/50] [349/375] loss: 572.186
[3/50] [350/375] loss: 574.073
[3/50] [351/375] loss: 575.721
[3/50] [352/375] loss: 577.360
[3/50] [353/375] loss: 578.583
[3/50] [354/375] loss: 580.122
[3/50] [355/375] loss: 581.644
[3/50] [

...

[47/50] [164/375] loss: 11.570
[47/50] [165/375] loss: 11.597
[47/50] [166/375] loss: 11.639
[47/50] [167/375] loss: 11.734
[47/50] [168/375] loss: 11.781
[47/50] [169/375] loss: 11.814
[47/50] [170/375] loss: 11.869
[47/50] [171/375] loss: 11.915
[47/50] [172/375] loss: 11.958
[47/50] [173/375] loss: 11.977
[47/50] [174/375] loss: 12.090
[47/50] [175/375] loss: 12.136
[47/50] [176/375] loss: 12.144
[47/50] [177/375] loss: 12.188
[47/50] [178/375] loss: 12.205
[47/50] [179/375] loss: 12.512
[47/50] [180/375] loss: 12.619
[47/50] [181/375] loss: 12.667
[47/50] [182/375] loss: 12.690
[47/50] [183/375] loss: 12.701
[47/50] [184/375] loss: 12.766
[47/50] [185/375] loss: 12.831
[47/50] [186/375] loss: 12.842
[47/50] [187/375] loss: 12.857
[47/50] [188/375] loss: 13.006
[47/50] [189/375] loss: 13.020
[47/50] [190/375] loss: 13.058
[47/50] [191/375] loss: 13.116
[47/50] [192/375] loss: 13.121
[47/50] [193/375] loss: 13.135
[47/50] [194/375] loss: 13.263
[47/50] [195/375] loss: 13.305
[47/50] 

[48/50] [57/375] loss: 3.880
[48/50] [58/375] loss: 3.901
[48/50] [59/375] loss: 3.988
[48/50] [60/375] loss: 4.181
[48/50] [61/375] loss: 4.190
[48/50] [62/375] loss: 4.258
[48/50] [63/375] loss: 4.286
[48/50] [64/375] loss: 4.288
[48/50] [65/375] loss: 4.314
[48/50] [66/375] loss: 4.323
[48/50] [67/375] loss: 4.325
[48/50] [68/375] loss: 4.363
[48/50] [69/375] loss: 4.371
[48/50] [70/375] loss: 4.376
[48/50] [71/375] loss: 4.414
[48/50] [72/375] loss: 4.429
[48/50] [73/375] loss: 4.437
[48/50] [74/375] loss: 4.483
[48/50] [75/375] loss: 4.522
[48/50] [76/375] loss: 4.601
[48/50] [77/375] loss: 4.648
[48/50] [78/375] loss: 4.664
[48/50] [79/375] loss: 4.671
[48/50] [80/375] loss: 4.893
[48/50] [81/375] loss: 4.964
[48/50] [82/375] loss: 5.055
[48/50] [83/375] loss: 5.098
[48/50] [84/375] loss: 5.122
[48/50] [85/375] loss: 5.215
[48/50] [86/375] loss: 5.265
[48/50] [87/375] loss: 5.470
[48/50] [88/375] loss: 5.544
[48/50] [89/375] loss: 5.577
[48/50] [90/375] loss: 5.591
[48/50] [91/37

[48/50] [326/375] loss: 23.938
[48/50] [327/375] loss: 23.971
[48/50] [328/375] loss: 24.074
[48/50] [329/375] loss: 24.130
[48/50] [330/375] loss: 24.213
[48/50] [331/375] loss: 24.283
[48/50] [332/375] loss: 24.299
[48/50] [333/375] loss: 24.323
[48/50] [334/375] loss: 24.547
[48/50] [335/375] loss: 24.639
[48/50] [336/375] loss: 24.649
[48/50] [337/375] loss: 24.658
[48/50] [338/375] loss: 24.671
[48/50] [339/375] loss: 24.712
[48/50] [340/375] loss: 24.730
[48/50] [341/375] loss: 24.940
[48/50] [342/375] loss: 25.405
[48/50] [343/375] loss: 25.417
[48/50] [344/375] loss: 25.488
[48/50] [345/375] loss: 25.519
[48/50] [346/375] loss: 25.613
[48/50] [347/375] loss: 25.621
[48/50] [348/375] loss: 25.636
[48/50] [349/375] loss: 25.655
[48/50] [350/375] loss: 25.673
[48/50] [351/375] loss: 25.685
[48/50] [352/375] loss: 25.703
[48/50] [353/375] loss: 25.749
[48/50] [354/375] loss: 25.860
[48/50] [355/375] loss: 26.058
[48/50] [356/375] loss: 26.343
[48/50] [357/375] loss: 26.375
[48/50] 

[49/50] [224/375] loss: 12.312
[49/50] [225/375] loss: 12.362
[49/50] [226/375] loss: 12.393
[49/50] [227/375] loss: 12.476
[49/50] [228/375] loss: 12.553
[49/50] [229/375] loss: 12.597
[49/50] [230/375] loss: 12.609
[49/50] [231/375] loss: 12.700
[49/50] [232/375] loss: 12.717
[49/50] [233/375] loss: 12.748
[49/50] [234/375] loss: 12.814
[49/50] [235/375] loss: 12.838
[49/50] [236/375] loss: 12.846
[49/50] [237/375] loss: 12.860
[49/50] [238/375] loss: 12.875
[49/50] [239/375] loss: 12.908
[49/50] [240/375] loss: 12.956
[49/50] [241/375] loss: 13.026
[49/50] [242/375] loss: 13.091
[49/50] [243/375] loss: 13.119
[49/50] [244/375] loss: 13.216
[49/50] [245/375] loss: 13.222
[49/50] [246/375] loss: 13.437
[49/50] [247/375] loss: 13.444
[49/50] [248/375] loss: 13.448
[49/50] [249/375] loss: 13.497
[49/50] [250/375] loss: 13.524
[49/50] [251/375] loss: 13.559
[49/50] [252/375] loss: 13.573
[49/50] [253/375] loss: 13.615
[49/50] [254/375] loss: 13.827
[49/50] [255/375] loss: 13.846
[49/50] 

[50/50] [120/375] loss: 5.109
[50/50] [121/375] loss: 5.112
[50/50] [122/375] loss: 5.126
[50/50] [123/375] loss: 5.211
[50/50] [124/375] loss: 5.227
[50/50] [125/375] loss: 5.236
[50/50] [126/375] loss: 5.254
[50/50] [127/375] loss: 5.264
[50/50] [128/375] loss: 5.275
[50/50] [129/375] loss: 5.295
[50/50] [130/375] loss: 5.312
[50/50] [131/375] loss: 5.316
[50/50] [132/375] loss: 5.331
[50/50] [133/375] loss: 5.569
[50/50] [134/375] loss: 5.701
[50/50] [135/375] loss: 5.734
[50/50] [136/375] loss: 5.742
[50/50] [137/375] loss: 5.765
[50/50] [138/375] loss: 5.881
[50/50] [139/375] loss: 5.905
[50/50] [140/375] loss: 5.994
[50/50] [141/375] loss: 6.002
[50/50] [142/375] loss: 6.019
[50/50] [143/375] loss: 6.026
[50/50] [144/375] loss: 6.087
[50/50] [145/375] loss: 6.142
[50/50] [146/375] loss: 6.294
[50/50] [147/375] loss: 6.316
[50/50] [148/375] loss: 6.334
[50/50] [149/375] loss: 6.353
[50/50] [150/375] loss: 6.363
[50/50] [151/375] loss: 6.377
[50/50] [152/375] loss: 6.392
[50/50] [1

In [32]:
resnet18_pretrain=models.resnet18(pretrained=True)
resnet18_pretrain.fc=nn.Linear(512,num_classes)
resnet18_pretrain=resnet18_pretrain.to(device)
optimizer=optim.SGD(resnet18_pretrain.parameters(),lr=lr,momentum=0.9,weight_decay=0.0005)
train(resnet18_pretrain,train_dataloader,optimizer)

[1/23] [0/375] loss: 1.874
[1/23] [1/375] loss: 3.682
[1/23] [2/375] loss: 5.762
[1/23] [3/375] loss: 7.507
[1/23] [4/375] loss: 9.411
[1/23] [5/375] loss: 11.655
[1/23] [6/375] loss: 13.509
[1/23] [7/375] loss: 15.661
[1/23] [8/375] loss: 17.353
[1/23] [9/375] loss: 19.123
[1/23] [10/375] loss: 21.057
[1/23] [11/375] loss: 22.814
[1/23] [12/375] loss: 24.765
[1/23] [13/375] loss: 26.616
[1/23] [14/375] loss: 28.590
[1/23] [15/375] loss: 30.000
[1/23] [16/375] loss: 32.258
[1/23] [17/375] loss: 34.170
[1/23] [18/375] loss: 36.336
[1/23] [19/375] loss: 37.736
[1/23] [20/375] loss: 40.583
[1/23] [21/375] loss: 43.708
[1/23] [22/375] loss: 45.783
[1/23] [23/375] loss: 47.869
[1/23] [24/375] loss: 50.380
[1/23] [25/375] loss: 52.638
[1/23] [26/375] loss: 54.529
[1/23] [27/375] loss: 57.333
[1/23] [28/375] loss: 59.676
[1/23] [29/375] loss: 61.755
[1/23] [30/375] loss: 63.968
[1/23] [31/375] loss: 66.185
[1/23] [32/375] loss: 68.353
[1/23] [33/375] loss: 70.684
[1/23] [34/375] loss: 72.649


[1/23] [270/375] loss: 506.182
[1/23] [271/375] loss: 508.009
[1/23] [272/375] loss: 509.499
[1/23] [273/375] loss: 510.698
[1/23] [274/375] loss: 512.556
[1/23] [275/375] loss: 514.655
[1/23] [276/375] loss: 516.086
[1/23] [277/375] loss: 517.434
[1/23] [278/375] loss: 519.589
[1/23] [279/375] loss: 521.338
[1/23] [280/375] loss: 522.589
[1/23] [281/375] loss: 524.342
[1/23] [282/375] loss: 525.813
[1/23] [283/375] loss: 527.319
[1/23] [284/375] loss: 529.041
[1/23] [285/375] loss: 530.675
[1/23] [286/375] loss: 532.816
[1/23] [287/375] loss: 534.597
[1/23] [288/375] loss: 536.841
[1/23] [289/375] loss: 538.341
[1/23] [290/375] loss: 540.655
[1/23] [291/375] loss: 541.925
[1/23] [292/375] loss: 543.368
[1/23] [293/375] loss: 544.788
[1/23] [294/375] loss: 546.719
[1/23] [295/375] loss: 548.402
[1/23] [296/375] loss: 550.421
[1/23] [297/375] loss: 552.050
[1/23] [298/375] loss: 553.465
[1/23] [299/375] loss: 554.930
[1/23] [300/375] loss: 557.044
[1/23] [301/375] loss: 558.563
[1/23] [

[2/23] [165/375] loss: 245.896
[2/23] [166/375] loss: 247.456
[2/23] [167/375] loss: 248.925
[2/23] [168/375] loss: 250.066
[2/23] [169/375] loss: 251.535
[2/23] [170/375] loss: 253.102
[2/23] [171/375] loss: 255.123
[2/23] [172/375] loss: 256.898
[2/23] [173/375] loss: 258.465
[2/23] [174/375] loss: 259.920
[2/23] [175/375] loss: 261.469
[2/23] [176/375] loss: 263.025
[2/23] [177/375] loss: 264.090
[2/23] [178/375] loss: 265.800
[2/23] [179/375] loss: 267.221
[2/23] [180/375] loss: 268.561
[2/23] [181/375] loss: 269.581
[2/23] [182/375] loss: 270.917
[2/23] [183/375] loss: 272.243
[2/23] [184/375] loss: 273.255
[2/23] [185/375] loss: 274.931
[2/23] [186/375] loss: 276.688
[2/23] [187/375] loss: 278.224
[2/23] [188/375] loss: 279.465
[2/23] [189/375] loss: 280.943
[2/23] [190/375] loss: 282.382
[2/23] [191/375] loss: 283.366
[2/23] [192/375] loss: 284.619
[2/23] [193/375] loss: 285.765
[2/23] [194/375] loss: 287.367
[2/23] [195/375] loss: 288.968
[2/23] [196/375] loss: 290.147
[2/23] [

[3/23] [58/375] loss: 71.595
[3/23] [59/375] loss: 72.605
[3/23] [60/375] loss: 73.844
[3/23] [61/375] loss: 75.048
[3/23] [62/375] loss: 76.012
[3/23] [63/375] loss: 77.377
[3/23] [64/375] loss: 78.485
[3/23] [65/375] loss: 79.349
[3/23] [66/375] loss: 80.630
[3/23] [67/375] loss: 81.959
[3/23] [68/375] loss: 82.832
[3/23] [69/375] loss: 83.977
[3/23] [70/375] loss: 85.131
[3/23] [71/375] loss: 86.327
[3/23] [72/375] loss: 87.794
[3/23] [73/375] loss: 89.152
[3/23] [74/375] loss: 90.390
[3/23] [75/375] loss: 91.911
[3/23] [76/375] loss: 92.844
[3/23] [77/375] loss: 93.991
[3/23] [78/375] loss: 94.839
[3/23] [79/375] loss: 96.311
[3/23] [80/375] loss: 97.234
[3/23] [81/375] loss: 98.937
[3/23] [82/375] loss: 100.547
[3/23] [83/375] loss: 101.821
[3/23] [84/375] loss: 102.803
[3/23] [85/375] loss: 104.151
[3/23] [86/375] loss: 105.400
[3/23] [87/375] loss: 106.407
[3/23] [88/375] loss: 108.341
[3/23] [89/375] loss: 109.243
[3/23] [90/375] loss: 110.541
[3/23] [91/375] loss: 111.915
[3/2

[3/23] [325/375] loss: 392.830
[3/23] [326/375] loss: 393.658
[3/23] [327/375] loss: 394.458
[3/23] [328/375] loss: 395.587
[3/23] [329/375] loss: 396.808
[3/23] [330/375] loss: 397.866
[3/23] [331/375] loss: 399.454
[3/23] [332/375] loss: 401.413
[3/23] [333/375] loss: 402.590
[3/23] [334/375] loss: 404.182
[3/23] [335/375] loss: 405.378
[3/23] [336/375] loss: 406.592
[3/23] [337/375] loss: 407.928
[3/23] [338/375] loss: 408.858
[3/23] [339/375] loss: 409.810
[3/23] [340/375] loss: 411.007
[3/23] [341/375] loss: 412.327
[3/23] [342/375] loss: 413.428
[3/23] [343/375] loss: 414.117
[3/23] [344/375] loss: 414.955
[3/23] [345/375] loss: 416.285
[3/23] [346/375] loss: 417.480
[3/23] [347/375] loss: 418.591
[3/23] [348/375] loss: 419.829
[3/23] [349/375] loss: 421.269
[3/23] [350/375] loss: 422.445
[3/23] [351/375] loss: 423.491
[3/23] [352/375] loss: 424.660
[3/23] [353/375] loss: 425.687
[3/23] [354/375] loss: 426.304
[3/23] [355/375] loss: 427.192
[3/23] [356/375] loss: 428.237
[3/23] [

...

[21/23] [1/375] loss: 0.112
[21/23] [2/375] loss: 0.126
[21/23] [3/375] loss: 0.158
[21/23] [4/375] loss: 0.283
[21/23] [5/375] loss: 0.595
[21/23] [6/375] loss: 0.826
[21/23] [7/375] loss: 1.036
[21/23] [8/375] loss: 1.082
[21/23] [9/375] loss: 1.087
[21/23] [10/375] loss: 1.224
[21/23] [11/375] loss: 1.242
[21/23] [12/375] loss: 1.338
[21/23] [13/375] loss: 1.453
[21/23] [14/375] loss: 1.491
[21/23] [15/375] loss: 1.535
[21/23] [16/375] loss: 1.590
[21/23] [17/375] loss: 1.595
[21/23] [18/375] loss: 1.632
[21/23] [19/375] loss: 1.797
[21/23] [20/375] loss: 1.910
[21/23] [21/375] loss: 2.110
[21/23] [22/375] loss: 2.200
[21/23] [23/375] loss: 2.288
[21/23] [24/375] loss: 2.312
[21/23] [25/375] loss: 2.319
[21/23] [26/375] loss: 2.396
[21/23] [27/375] loss: 2.777
[21/23] [28/375] loss: 2.796
[21/23] [29/375] loss: 2.804
[21/23] [30/375] loss: 2.962
[21/23] [31/375] loss: 3.091
[21/23] [32/375] loss: 3.195
[21/23] [33/375] loss: 3.216
[21/23] [34/375] loss: 3.224
[21/23] [35/375] loss: 

[21/23] [272/375] loss: 34.569
[21/23] [273/375] loss: 34.756
[21/23] [274/375] loss: 34.934
[21/23] [275/375] loss: 35.063
[21/23] [276/375] loss: 35.252
[21/23] [277/375] loss: 35.260
[21/23] [278/375] loss: 35.467
[21/23] [279/375] loss: 35.749
[21/23] [280/375] loss: 35.773
[21/23] [281/375] loss: 35.790
[21/23] [282/375] loss: 35.862
[21/23] [283/375] loss: 35.880
[21/23] [284/375] loss: 35.960
[21/23] [285/375] loss: 36.011
[21/23] [286/375] loss: 36.520
[21/23] [287/375] loss: 36.652
[21/23] [288/375] loss: 36.747
[21/23] [289/375] loss: 36.821
[21/23] [290/375] loss: 36.913
[21/23] [291/375] loss: 37.265
[21/23] [292/375] loss: 37.306
[21/23] [293/375] loss: 37.496
[21/23] [294/375] loss: 37.545
[21/23] [295/375] loss: 37.625
[21/23] [296/375] loss: 37.649
[21/23] [297/375] loss: 37.687
[21/23] [298/375] loss: 38.075
[21/23] [299/375] loss: 38.383
[21/23] [300/375] loss: 38.422
[21/23] [301/375] loss: 38.430
[21/23] [302/375] loss: 38.668
[21/23] [303/375] loss: 38.827
[21/23] 

[22/23] [168/375] loss: 13.212
[22/23] [169/375] loss: 13.240
[22/23] [170/375] loss: 13.803
[22/23] [171/375] loss: 14.127
[22/23] [172/375] loss: 14.132
[22/23] [173/375] loss: 14.208
[22/23] [174/375] loss: 14.405
[22/23] [175/375] loss: 14.557
[22/23] [176/375] loss: 14.708
[22/23] [177/375] loss: 14.775
[22/23] [178/375] loss: 14.796
[22/23] [179/375] loss: 14.819
[22/23] [180/375] loss: 14.824
[22/23] [181/375] loss: 15.163
[22/23] [182/375] loss: 15.399
[22/23] [183/375] loss: 15.411
[22/23] [184/375] loss: 15.419
[22/23] [185/375] loss: 15.428
[22/23] [186/375] loss: 15.565
[22/23] [187/375] loss: 15.608
[22/23] [188/375] loss: 15.635
[22/23] [189/375] loss: 15.731
[22/23] [190/375] loss: 15.760
[22/23] [191/375] loss: 15.809
[22/23] [192/375] loss: 16.118
[22/23] [193/375] loss: 16.194
[22/23] [194/375] loss: 16.227
[22/23] [195/375] loss: 16.418
[22/23] [196/375] loss: 16.498
[22/23] [197/375] loss: 16.589
[22/23] [198/375] loss: 16.780
[22/23] [199/375] loss: 16.809
[22/23] 

[23/23] [61/375] loss: 5.235
[23/23] [62/375] loss: 5.601
[23/23] [63/375] loss: 5.795
[23/23] [64/375] loss: 5.941
[23/23] [65/375] loss: 5.950
[23/23] [66/375] loss: 6.001
[23/23] [67/375] loss: 6.017
[23/23] [68/375] loss: 6.025
[23/23] [69/375] loss: 6.040
[23/23] [70/375] loss: 6.068
[23/23] [71/375] loss: 6.094
[23/23] [72/375] loss: 6.169
[23/23] [73/375] loss: 6.294
[23/23] [74/375] loss: 6.303
[23/23] [75/375] loss: 6.313
[23/23] [76/375] loss: 6.444
[23/23] [77/375] loss: 6.526
[23/23] [78/375] loss: 6.539
[23/23] [79/375] loss: 6.910
[23/23] [80/375] loss: 7.109
[23/23] [81/375] loss: 7.133
[23/23] [82/375] loss: 7.240
[23/23] [83/375] loss: 7.324
[23/23] [84/375] loss: 7.357
[23/23] [85/375] loss: 7.419
[23/23] [86/375] loss: 7.532
[23/23] [87/375] loss: 7.568
[23/23] [88/375] loss: 7.574
[23/23] [89/375] loss: 7.602
[23/23] [90/375] loss: 7.606
[23/23] [91/375] loss: 7.629
[23/23] [92/375] loss: 7.702
[23/23] [93/375] loss: 7.787
[23/23] [94/375] loss: 7.952
[23/23] [95/37

[23/23] [329/375] loss: 31.217
[23/23] [330/375] loss: 31.220
[23/23] [331/375] loss: 31.266
[23/23] [332/375] loss: 31.270
[23/23] [333/375] loss: 31.297
[23/23] [334/375] loss: 31.302
[23/23] [335/375] loss: 31.342
[23/23] [336/375] loss: 31.596
[23/23] [337/375] loss: 31.714
[23/23] [338/375] loss: 31.754
[23/23] [339/375] loss: 31.755
[23/23] [340/375] loss: 31.876
[23/23] [341/375] loss: 31.979
[23/23] [342/375] loss: 32.026
[23/23] [343/375] loss: 32.051
[23/23] [344/375] loss: 32.127
[23/23] [345/375] loss: 32.155
[23/23] [346/375] loss: 32.232
[23/23] [347/375] loss: 32.604
[23/23] [348/375] loss: 32.610
[23/23] [349/375] loss: 32.665
[23/23] [350/375] loss: 32.680
[23/23] [351/375] loss: 32.785
[23/23] [352/375] loss: 32.825
[23/23] [353/375] loss: 32.982
[23/23] [354/375] loss: 33.041
[23/23] [355/375] loss: 33.098
[23/23] [356/375] loss: 33.151
[23/23] [357/375] loss: 33.317
[23/23] [358/375] loss: 33.362
[23/23] [359/375] loss: 33.392
[23/23] [360/375] loss: 33.400
[23/23] 

In [10]:
torch.cuda.empty_cache()
vgg16 = models.vgg16(pretrained=True)
vgg16.classifier=nn.Sequential(
    nn.Linear(25088,4096),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(4096,4096),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(4096,num_classes),
)
vgg16=vgg16.to(device)
optimizer=optim.SGD(vgg16.parameters(),lr=lr,momentum=0.9,weight_decay=0.0005)
train(vgg16,train_dataloader,optimizer)

[1/50] [0/375] loss: 1.800
[1/50] [1/375] loss: 3.592
[1/50] [2/375] loss: 5.404
[1/50] [3/375] loss: 7.225
[1/50] [4/375] loss: 9.088
[1/50] [5/375] loss: 10.909
[1/50] [6/375] loss: 12.736
[1/50] [7/375] loss: 14.527
[1/50] [8/375] loss: 16.357
[1/50] [9/375] loss: 18.140
[1/50] [10/375] loss: 19.940
[1/50] [11/375] loss: 21.743
[1/50] [12/375] loss: 23.531
[1/50] [13/375] loss: 25.389
[1/50] [14/375] loss: 27.175
[1/50] [15/375] loss: 28.955
[1/50] [16/375] loss: 30.795
[1/50] [17/375] loss: 32.540
[1/50] [18/375] loss: 34.340
[1/50] [19/375] loss: 36.153
[1/50] [20/375] loss: 37.951
[1/50] [21/375] loss: 39.757
[1/50] [22/375] loss: 41.519
[1/50] [23/375] loss: 43.290
[1/50] [24/375] loss: 45.113
[1/50] [25/375] loss: 46.849
[1/50] [26/375] loss: 48.601
[1/50] [27/375] loss: 50.372
[1/50] [28/375] loss: 52.159
[1/50] [29/375] loss: 53.928
[1/50] [30/375] loss: 55.693
[1/50] [31/375] loss: 57.505
[1/50] [32/375] loss: 59.322
[1/50] [33/375] loss: 61.129
[1/50] [34/375] loss: 62.934


[1/50] [270/375] loss: 467.069
[1/50] [271/375] loss: 468.832
[1/50] [272/375] loss: 470.563
[1/50] [273/375] loss: 472.287
[1/50] [274/375] loss: 473.904
[1/50] [275/375] loss: 475.595
[1/50] [276/375] loss: 477.432
[1/50] [277/375] loss: 479.177
[1/50] [278/375] loss: 480.937
[1/50] [279/375] loss: 482.691
[1/50] [280/375] loss: 484.520
[1/50] [281/375] loss: 486.130
[1/50] [282/375] loss: 487.923
[1/50] [283/375] loss: 489.662
[1/50] [284/375] loss: 491.210
[1/50] [285/375] loss: 492.997
[1/50] [286/375] loss: 494.480
[1/50] [287/375] loss: 496.095
[1/50] [288/375] loss: 497.725
[1/50] [289/375] loss: 499.512
[1/50] [290/375] loss: 501.210
[1/50] [291/375] loss: 502.748
[1/50] [292/375] loss: 504.545
[1/50] [293/375] loss: 506.115
[1/50] [294/375] loss: 507.639
[1/50] [295/375] loss: 509.044
[1/50] [296/375] loss: 510.805
[1/50] [297/375] loss: 512.554
[1/50] [298/375] loss: 514.198
[1/50] [299/375] loss: 515.904
[1/50] [300/375] loss: 517.498
[1/50] [301/375] loss: 519.207
[1/50] [

[2/50] [165/375] loss: 265.959
[2/50] [166/375] loss: 267.589
[2/50] [167/375] loss: 269.125
[2/50] [168/375] loss: 270.793
[2/50] [169/375] loss: 272.388
[2/50] [170/375] loss: 274.004
[2/50] [171/375] loss: 275.532
[2/50] [172/375] loss: 277.014
[2/50] [173/375] loss: 278.594
[2/50] [174/375] loss: 280.190
[2/50] [175/375] loss: 281.779
[2/50] [176/375] loss: 283.279
[2/50] [177/375] loss: 284.849
[2/50] [178/375] loss: 286.216
[2/50] [179/375] loss: 287.819
[2/50] [180/375] loss: 289.227
[2/50] [181/375] loss: 290.608
[2/50] [182/375] loss: 292.199
[2/50] [183/375] loss: 294.027
[2/50] [184/375] loss: 295.557
[2/50] [185/375] loss: 297.264
[2/50] [186/375] loss: 298.937
[2/50] [187/375] loss: 300.618
[2/50] [188/375] loss: 302.269
[2/50] [189/375] loss: 303.951
[2/50] [190/375] loss: 305.629
[2/50] [191/375] loss: 307.394
[2/50] [192/375] loss: 308.788
[2/50] [193/375] loss: 310.202
[2/50] [194/375] loss: 311.816
[2/50] [195/375] loss: 313.683
[2/50] [196/375] loss: 315.179
[2/50] [

[3/50] [58/375] loss: 86.111
[3/50] [59/375] loss: 87.219
[3/50] [60/375] loss: 89.251
[3/50] [61/375] loss: 90.728
[3/50] [62/375] loss: 92.474
[3/50] [63/375] loss: 93.950
[3/50] [64/375] loss: 95.276
[3/50] [65/375] loss: 96.826
[3/50] [66/375] loss: 98.165
[3/50] [67/375] loss: 99.689
[3/50] [68/375] loss: 100.977
[3/50] [69/375] loss: 102.194
[3/50] [70/375] loss: 103.370
[3/50] [71/375] loss: 104.672
[3/50] [72/375] loss: 106.183
[3/50] [73/375] loss: 107.475
[3/50] [74/375] loss: 108.588
[3/50] [75/375] loss: 110.061
[3/50] [76/375] loss: 112.059
[3/50] [77/375] loss: 113.242
[3/50] [78/375] loss: 114.747
[3/50] [79/375] loss: 116.208
[3/50] [80/375] loss: 117.522
[3/50] [81/375] loss: 119.169
[3/50] [82/375] loss: 120.535
[3/50] [83/375] loss: 121.854
[3/50] [84/375] loss: 123.034
[3/50] [85/375] loss: 124.835
[3/50] [86/375] loss: 126.502
[3/50] [87/375] loss: 128.010
[3/50] [88/375] loss: 129.747
[3/50] [89/375] loss: 131.184
[3/50] [90/375] loss: 132.569
[3/50] [91/375] loss

[3/50] [324/375] loss: 477.251
[3/50] [325/375] loss: 478.377
[3/50] [326/375] loss: 479.904
[3/50] [327/375] loss: 480.985
[3/50] [328/375] loss: 482.393
[3/50] [329/375] loss: 484.015
[3/50] [330/375] loss: 485.563
[3/50] [331/375] loss: 487.153
[3/50] [332/375] loss: 488.665
[3/50] [333/375] loss: 489.974
[3/50] [334/375] loss: 491.536
[3/50] [335/375] loss: 493.006
[3/50] [336/375] loss: 494.550
[3/50] [337/375] loss: 495.610
[3/50] [338/375] loss: 496.750
[3/50] [339/375] loss: 498.362
[3/50] [340/375] loss: 499.583
[3/50] [341/375] loss: 500.799
[3/50] [342/375] loss: 502.347
[3/50] [343/375] loss: 503.664
[3/50] [344/375] loss: 504.679
[3/50] [345/375] loss: 506.154
[3/50] [346/375] loss: 507.310
[3/50] [347/375] loss: 508.697
[3/50] [348/375] loss: 510.192
[3/50] [349/375] loss: 511.253
[3/50] [350/375] loss: 512.404
[3/50] [351/375] loss: 513.591
[3/50] [352/375] loss: 514.452
[3/50] [353/375] loss: 515.711
[3/50] [354/375] loss: 516.972
[3/50] [355/375] loss: 518.386
[3/50] [

...

[48/50] [35/375] loss: 5.520
[48/50] [36/375] loss: 5.583
[48/50] [37/375] loss: 5.602
[48/50] [38/375] loss: 5.676
[48/50] [39/375] loss: 5.688
[48/50] [40/375] loss: 5.864
[48/50] [41/375] loss: 5.905
[48/50] [42/375] loss: 5.921
[48/50] [43/375] loss: 5.996
[48/50] [44/375] loss: 6.465
[48/50] [45/375] loss: 6.567
[48/50] [46/375] loss: 6.598
[48/50] [47/375] loss: 6.845
[48/50] [48/375] loss: 6.869
[48/50] [49/375] loss: 7.145
[48/50] [50/375] loss: 7.619
[48/50] [51/375] loss: 7.674
[48/50] [52/375] loss: 7.684
[48/50] [53/375] loss: 7.775
[48/50] [54/375] loss: 8.217
[48/50] [55/375] loss: 8.312
[48/50] [56/375] loss: 8.406
[48/50] [57/375] loss: 8.462
[48/50] [58/375] loss: 8.769
[48/50] [59/375] loss: 9.384
[48/50] [60/375] loss: 9.725
[48/50] [61/375] loss: 9.746
[48/50] [62/375] loss: 10.141
[48/50] [63/375] loss: 10.263
[48/50] [64/375] loss: 10.470
[48/50] [65/375] loss: 10.653
[48/50] [66/375] loss: 10.662
[48/50] [67/375] loss: 10.772
[48/50] [68/375] loss: 10.795
[48/50]

[48/50] [303/375] loss: 35.769
[48/50] [304/375] loss: 35.925
[48/50] [305/375] loss: 35.935
[48/50] [306/375] loss: 35.945
[48/50] [307/375] loss: 35.947
[48/50] [308/375] loss: 36.016
[48/50] [309/375] loss: 36.138
[48/50] [310/375] loss: 36.311
[48/50] [311/375] loss: 36.330
[48/50] [312/375] loss: 36.348
[48/50] [313/375] loss: 36.380
[48/50] [314/375] loss: 36.432
[48/50] [315/375] loss: 36.466
[48/50] [316/375] loss: 36.760
[48/50] [317/375] loss: 36.763
[48/50] [318/375] loss: 36.827
[48/50] [319/375] loss: 37.256
[48/50] [320/375] loss: 37.453
[48/50] [321/375] loss: 37.531
[48/50] [322/375] loss: 37.725
[48/50] [323/375] loss: 37.743
[48/50] [324/375] loss: 37.916
[48/50] [325/375] loss: 38.175
[48/50] [326/375] loss: 38.264
[48/50] [327/375] loss: 38.436
[48/50] [328/375] loss: 38.474
[48/50] [329/375] loss: 38.486
[48/50] [330/375] loss: 38.523
[48/50] [331/375] loss: 38.595
[48/50] [332/375] loss: 38.674
[48/50] [333/375] loss: 38.779
[48/50] [334/375] loss: 39.003
[48/50] 

[49/50] [198/375] loss: 23.265
[49/50] [199/375] loss: 23.297
[49/50] [200/375] loss: 23.302
[49/50] [201/375] loss: 23.542
[49/50] [202/375] loss: 23.885
[49/50] [203/375] loss: 23.965
[49/50] [204/375] loss: 24.251
[49/50] [205/375] loss: 24.494
[49/50] [206/375] loss: 24.566
[49/50] [207/375] loss: 25.000
[49/50] [208/375] loss: 25.077
[49/50] [209/375] loss: 25.164
[49/50] [210/375] loss: 25.471
[49/50] [211/375] loss: 26.166
[49/50] [212/375] loss: 26.355
[49/50] [213/375] loss: 26.496
[49/50] [214/375] loss: 26.722
[49/50] [215/375] loss: 26.884
[49/50] [216/375] loss: 27.007
[49/50] [217/375] loss: 27.043
[49/50] [218/375] loss: 27.295
[49/50] [219/375] loss: 27.696
[49/50] [220/375] loss: 28.056
[49/50] [221/375] loss: 28.149
[49/50] [222/375] loss: 28.272
[49/50] [223/375] loss: 28.456
[49/50] [224/375] loss: 28.497
[49/50] [225/375] loss: 28.739
[49/50] [226/375] loss: 28.769
[49/50] [227/375] loss: 28.886
[49/50] [228/375] loss: 28.978
[49/50] [229/375] loss: 28.994
[49/50] 

[50/50] [93/375] loss: 7.171
[50/50] [94/375] loss: 7.202
[50/50] [95/375] loss: 7.293
[50/50] [96/375] loss: 7.323
[50/50] [97/375] loss: 7.402
[50/50] [98/375] loss: 7.411
[50/50] [99/375] loss: 7.417
[50/50] [100/375] loss: 7.421
[50/50] [101/375] loss: 7.472
[50/50] [102/375] loss: 7.560
[50/50] [103/375] loss: 7.626
[50/50] [104/375] loss: 7.681
[50/50] [105/375] loss: 7.682
[50/50] [106/375] loss: 7.688
[50/50] [107/375] loss: 7.697
[50/50] [108/375] loss: 7.755
[50/50] [109/375] loss: 7.808
[50/50] [110/375] loss: 8.014
[50/50] [111/375] loss: 8.029
[50/50] [112/375] loss: 8.030
[50/50] [113/375] loss: 8.033
[50/50] [114/375] loss: 8.049
[50/50] [115/375] loss: 8.130
[50/50] [116/375] loss: 8.134
[50/50] [117/375] loss: 8.187
[50/50] [118/375] loss: 8.201
[50/50] [119/375] loss: 8.234
[50/50] [120/375] loss: 8.258
[50/50] [121/375] loss: 8.261
[50/50] [122/375] loss: 8.379
[50/50] [123/375] loss: 8.383
[50/50] [124/375] loss: 8.422
[50/50] [125/375] loss: 8.660
[50/50] [126/375]

[50/50] [360/375] loss: 28.709
[50/50] [361/375] loss: 29.162
[50/50] [362/375] loss: 29.176
[50/50] [363/375] loss: 29.182
[50/50] [364/375] loss: 29.185
[50/50] [365/375] loss: 29.210
[50/50] [366/375] loss: 29.648
[50/50] [367/375] loss: 29.974
[50/50] [368/375] loss: 30.525
[50/50] [369/375] loss: 30.579
[50/50] [370/375] loss: 30.630
[50/50] [371/375] loss: 30.733
[50/50] [372/375] loss: 30.738
[50/50] [373/375] loss: 30.760
[50/50] [374/375] loss: 30.805
Train accuracy: 97.383%


In [16]:
def test(model,dataloader):
    res=[]
    for data in dataloader:
        data=data.to(device)
        outputs=model(data)
        _, predict = torch.max(outputs.data, 1)
        predict=predict.tolist()
        res+=predict
    return res

testset=ICHDataset('TestingData',train=False)
dataloader=DataLoader(testset,batch_size,shuffle=False)
res=test(resnet18,dataloader)
print(res)

[1, 1, 5, 1, 3, 2, 5, 0, 3, 5, 0, 1, 2, 3, 0, 4, 4, 0, 1, 3, 0, 4, 0, 3, 5, 3, 5, 1, 5, 5, 0, 1, 4, 3, 0, 0, 3, 5, 1, 0, 4, 1, 3, 1, 4, 4, 3, 1, 5, 5, 2, 3, 5, 0, 1, 2, 4, 0, 2, 0, 4, 5, 3, 3, 4, 5, 3, 2, 0, 3, 5, 0, 0, 2, 5, 3, 5, 4, 5, 4, 3, 0, 4, 3, 5, 5, 5, 3, 0, 4, 1, 4, 1, 3, 0, 2, 1, 3, 0, 5, 2, 4, 0, 4, 0, 0, 4, 2, 1, 3, 1, 3, 5, 0, 2, 4, 3, 1, 4, 0, 2, 3, 1, 1, 0, 5, 3, 0, 4, 3, 5, 2, 4, 0, 0, 5, 3, 0, 0, 5, 3, 0, 4, 0, 4, 4, 4, 2, 3, 5, 3, 3, 4, 5, 0, 3, 0, 3, 0, 3, 3, 3, 0, 2, 5, 2, 3, 5, 0, 5, 5, 3, 4, 1, 1, 3, 3, 4, 2, 5, 0, 2, 1, 5, 1, 0, 4, 3, 2, 5, 4, 3, 3, 3, 0, 4, 1, 3, 2, 4, 0, 2, 0, 4, 4, 1, 5, 3, 0, 0, 2, 1, 0, 0, 5, 2, 5, 4, 0, 0, 2, 3, 3, 3, 3, 4, 1, 2, 1, 0, 0, 0, 3, 0, 1, 2, 0, 3, 0, 3, 1, 1, 0, 0, 2, 4, 0, 1, 5, 1, 2, 0, 3, 4, 3, 3, 3, 2, 0, 5, 3, 0, 1, 4, 4, 5, 1, 2, 0, 2, 3, 3, 1, 3, 5, 1, 0, 2, 3, 2, 5, 4, 2, 3, 0, 4, 2, 0, 3, 1, 1, 3, 5, 2, 0, 4, 4, 1, 0, 3, 3, 5, 2, 3, 1, 1, 0, 5, 0, 1, 0, 3, 2, 3, 0, 1, 3, 2, 5, 4, 0, 0, 4, 3, 4, 0, 0, 4, 1, 2, 0, 2, 4, 

In [17]:
# write excel file
wb=Workbook()
ws=wb.active
for i in range(len(testset)):
    val=testset.filename_list[i].split('/')[1]
    val=val.split('.')[0]
    ws.cell(i+1,1).value=val
    ws.cell(i+1,2).value=class_dict[res[i]]
wb.save('testing_result.xlsx')